In [38]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/Dan/upload/src/")

# Data path
import pathlib
DATA_FOLDER = pathlib.Path("/home/jovyan/work/Dan/data")
TABLE_PATH = DATA_FOLDER / 'pwdb/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = DATA_FOLDER / 'pwdb/word2vec/law2vec/Law2Vec.200d.txt'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.svm import SVC

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

In [39]:
# Read data
df = pd.read_pickle(TABLE_PATH)
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Category']

## law2vec model

In [41]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [42]:
class MeanEmbeddingVectorizer(object):
    """Calculate the mean of each word"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(l2v_dict))])
        else:
            self.dim=0

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0) for words in X]
        )

In [43]:
# Use pipes to implement steps of fit and transform method
svm_l2v = Pipeline([
    # Add the words we want to mean
    ("law2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use SVC algorithm
    ("SVM", SVC())])

## Train SVM Model

In [44]:
# split data into test and train sets
# where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=42, test_size=0.3, shuffle=True)

In [45]:
# Fit SVM model with out train data
svm_l2v.fit(X_train, y_train)

Pipeline(steps=[('law2vec vectorizer',
                 <__main__.MeanEmbeddingVectorizer object at 0x7fa8ea5c78b0>),
                ('SVM', SVC())])

In [46]:
# SVM prediction based on test data
prediction = svm_l2v.predict(X_test)
prediction

array([0, 4, 0, 4, 3, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 4, 2, 3, 4, 0, 6, 0,
       0, 0, 0, 4, 0, 4, 2, 3, 7, 2, 2, 0, 0, 3, 4, 0, 4, 0, 5, 3, 5, 5,
       6, 4, 3, 3, 0, 3, 5, 3, 6, 5, 2, 6, 0, 6, 2, 0, 5, 0, 0, 4, 2, 0,
       0, 6, 3, 0, 4, 0, 0, 0, 3, 0, 2, 6, 2, 3, 2, 6, 0, 0, 0, 0, 0, 0,
       5, 0, 4, 0, 0, 5, 3, 5, 0, 0, 0, 0, 4, 0, 2, 4, 2, 0, 0, 3, 0, 5,
       4, 0, 0, 0, 2, 2, 2, 3, 0, 0, 0, 6, 0, 0, 4, 5, 6, 0, 2, 3, 0, 0,
       0, 0, 0, 4, 0, 0, 4, 0, 5, 3, 5, 3, 3, 0, 0, 0, 2, 0, 0, 2, 3, 2,
       2, 4, 0, 0, 5, 0, 0, 3, 6, 2, 2, 0, 3, 0, 2, 0, 4, 5, 3, 2, 3, 0,
       0, 0, 3, 0, 0, 5, 6, 0, 0, 0, 0, 0, 6, 0, 0, 3, 2, 7, 5, 5, 2, 0,
       3, 3, 3, 2, 0, 6, 2, 6, 5, 2, 0, 5, 4, 0, 0, 3, 0, 0, 4, 0, 0, 0,
       0, 0, 3, 3, 3, 0, 3, 0, 2, 5, 2, 0, 0, 0, 6, 3, 0, 5, 2, 0, 0, 2,
       3, 3, 0, 0, 0, 5, 0, 0, 0, 6, 3, 2, 2, 5, 3, 0, 4, 0, 2, 4, 4, 3,
       3, 0, 0, 0, 0, 2, 0, 2, 3, 3, 2, 2, 0, 2, 0, 0, 4])

## Evaluation

In [47]:
model_evaluation_metrics(y_test, prediction, 'Category')

,Evaluation Metrics,Category
0,Accuracy,0.583630
1,Precission,0.482955
2,Recall,0.420164
3,F1 Score,0.395788
4,Mean Absolute Error,1.683274
5,Mean Squared Error,8.665480
